<a href="https://colab.research.google.com/github/jordynjey/Aprendiendo_claculadora/blob/master/censpag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from pandas.core.reshape.encoding import Index
from numpy.core.numeric import indices
#Librerias de lectura y rutas
from os import path
from pathlib import Path
from zmq import ROUTER_RAW
from typing import Container
#librerias http
import requests
import re
#libreria de formateo de html
from bs4 import BeautifulSoup
import lxml.html as html
#--------librerias de lectura y datos----
import numpy as np
import pandas as pd
from pandas import ExcelWriter
#Librerias para pasar al Excel#
from openpyxl import load_workbook,Workbook
from openpyxl.styles import Font
from datetime import datetime

#Pagina de CENS
url= 'https://www.cens.com.co'

#------------------------Primero-----------------------------#
pagi_cens=requests.get(url)
#imprimir estructura de la pagina
#print(pagi_cens.text)
#pagi_cens.content
#-----------------Funciones-----------------------#
def agregar_links_etiquetas_a(atributo,lista):
  lista=[s.get('href') for s in atributo]

def impresion_lista(lista_im):
  for i in lista_im:
    print(i)

def agregar_datos(lista_recorre,lista_agrega):
  for i in lista_recorre:
    lista_agrega.append(i)
  return lista_agrega

def quitar_duplicados_lista(lista_com):
  sin_duplica_links=[]
  for dato in lista_com:
    if dato not in sin_duplica_links:
      sin_duplica_links.append(dato)
  print('Se eliminaron : '+  str(len(lista_com)-len(sin_duplica_links))+ ' duplicados')
  #lista_com=sin_duplica_links
  return sin_duplica_links

def arreglar_relativas(lista):
  cambios=0
  indice=0
  for i in lista:
    li=str(i)
    if str(li).startswith('/'):
      #print('Antes de cambiar  ' , li)
      indice=lista.index(i)
      cadena=url+li
      #print('Despues de cambiar', cadena)
      lista[indice]=cadena
      cambios+=1
  print('Cambios realizados: '+ str(cambios))
  return lista

def quitar_restriciones(lista,listarestr):
    lista2=[]
    for a in lista:
      if a not in listarestr:
        lista2.append(a)
    lista=lista2
    return lista


def quitar_restriciones_segundo_nvel(lista,listarestr):
  lista2=[]
  for a in lista:
    bolean=False
    bolean=a.startswith('javascript')# los java script
    bolean2=a.startswith('/Portals/')# Los documentos publicos para los usuarios
    bolean3=a.startswith('#tpt') # etiquetas de contenido
    bolean4=('@' in a)
    if bolean==False:
      if bolean2==False:
        if bolean3==False:
          if bolean4==False:
            if a not in listarestr:
              lista2.append(a)
  lista=lista2
  return lista

def validar_prefijos_lista_a_lista(elemento,lista_prefijos):
  for prefijo in lista_prefijos:
    if elemento.startswith(prefijo) or '@' in elemento:
      print(f"El elemento' {elemento} ' comienza con el prefijo '{prefijo}' y no es válido.")
      return True
  return False

def validar_prefijos_lista_a_dato(lista_principal,lista_prefijos):
  lista_no=[]
  lista_auxiliar=[]
  for ele in lista_principal:
    for prefijo in lista_prefijos:
      if ele.startswith(prefijo)==True:
        print(f"El elemento' {ele} ' comienza con el prefijo '{prefijo}' y no es válido.")
        lista_no.append(ele)
      else:
        lista_auxiliar.append(ele)
  lista_auxiliar=quitar_restriciones(lista_auxiliar,lista_no)
  lista_principal=lista_auxiliar
  return lista_principal

def validar_no_contenga_lista_a_dato(lista_principal,lista_no_conte):
  lista_no=[]
  lista_auxiliar=[]
  for ele in lista_principal:
    for con in lista_no_conte:
      if con in ele:
        print(f"El elemento' {ele} ' contiene la palabra y /o letras '{con}' y no es válido.")
        lista_no.append(ele)
      else:
        lista_auxiliar.append(ele)
  lista_auxiliar=quitar_restriciones(lista_auxiliar,lista_no)
  lista_principal=lista_auxiliar
  return lista_principal

#creación de sopa de pagina principal
soup=BeautifulSoup(pagi_cens.text,'lxml')
# Imprimir de pretiffy es
#print(soup_main.prettify())

#find traer el primer atributo con las condiciones que desee colocar
#find_all trae todos las etiquetas que solicite
atributo=soup.findAll('a')
atributo2=soup.find_all('div')
linksprueba=[se.get('href') for se in atributo]

#print(list(enumerate(atributo)))
#atributo=soup_main.find_all('a', attrs={"target":'_blank'})
#atributo=soup_main.find_all('a', attrs={"target":""})
#print(len(atributo))
#atributo= soup_main.find_all('href')
#Para saber numero de X etiquetas
len(soup.find_all(href=True))
lista_prefijos=['javascript','/Portals/','#tpt','#cursos','#collapseExample','./','../']
lista_no_debe_contener=['@','chec','edeq','facebook','twitter','youtube','yout','tiktok',
                        'instagram','drive','community','Documentos','essa','plan-anticorrupcion','linkedin',
                        'citaciones_y_avisos']
#------------------------Segundo-----------------------------#
#--limpiar la información para poderla recorrer sin errores--#
restricciones=["javascript:__doPostBack('dnn$dnnSearch$cmdSearch','')"]
#--Se lee archivo de restricciones y se unen con las que ya se tienen de manera tecnica--#
path="/content/sample_data/links_restringido.xlsx"
openfile=pd.read_excel(path,engine='openpyxl',sheet_name='Sheet')
#Restricciones tecnicas o que no se van tomar en cuenta
links_restringidos=['#','',None,'https://gobiernodigital.mintic.gov.co/portal/','https://www.tiktok.com/@censgrupoepm',
                    'https://www.linkedin.com/company/64512425/admin/','https://www.youtube.com/user/buenasenergiascens' ,
                    'https://twitter.com/CENSGrupoEPM?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor' ,
                    'https://www.instagram.com/censgrupoepm/?hl=es-la','https://apps.apple.com/co/app/cens-app-movil/id1357634972',
                    'https://play.google.com/store/apps/details?id=com.cens.app.movil&hl=es_CO&gl=US','018000522955','018000414115',
                    '(607)5824444','115','javascript:void(0);','javascript:void(0)','https://www.gov.co/home/','https://www.colombia.co/',
                    'https://gobiernodigital.mintic.gov.co/portal','/institucional/transparencia/normatividad','/terminos-y-condiciones',
                    '/institucional/proteccion-de-datos-personales',"javascript:__doPostBack('dnn$ctr703$Login$Login_DNN$cmdLogin','')",
                    '#content','comercial@cooguasimales.com','eventos@cooguasimales.com','talentohumano@cooguasimales.com','mailto:soporte.clientes@cens.com.co',
                    'comercial@cooguasimales.com','eventos@cooguasimales.com','talentohumano@cooguasimales.com','https://www.veolia.com/latamib/es',
                    'https://biogersaesp.com/','https://www.coomultrasan.com.co/','https://www.ecoambientaldelnorte.com/','https://electriya.com/',
                    'https://www.almacenesrayco.com/','https://web.facebook.com/Electrogases-SAS-2333394940231027/','https://emzulia.com/','https://geoaseo.co/',
                    'https://www.cucuta.losolivos.co/',' portals=','https://www.vur.gov.co/','https://orglaesperanza.com/','http://www.suin-juriscol.gov.co/',
                    'http://www.strattus.com.co','https://www.cgm.gov.co/cgm/Paginaweb/SQR/Paginas/Radique-aqu%C3%AD-su-solicitud.aspx','https://www.contraloria.gov.co/es/web/guest/atencion-al-ciudadano/denuncias-y-otras-solicitudes-pqrd',
                    'https://www.procuraduria.gov.co/portal/pqrsdf.page','http://www.personcucutanortedesantander.gov.co/','https://www.superservicios.gov.co/Atenci%C3%B3n-y-servicios-a-la-ciudadan%C3%ADa/Tr%C3%A1mites-y-servicios',
                    'http://www.fiscalia.gov.co/colombia/servicios-de-informacion-al-ciudadano/buzon-de-quejas-y-reclamos/','https://www.contaduria.gov.co/peticiones-quejas-reclamos-sugerencias-y-denuncias',
                    'https://www.supersociedades.gov.co/modulo-pqrsd','http://www.secretariatransparencia.gov.co/','https://secure.ethicspoint.eu/domain/media/en/gui/104668/index.html',
                    'https://medellin.gestiontransparente.com/Rendicion/inicio.aspx','http://www.emvarias.com.co/default.aspx','https://wa.link/9xbvbv','https://solicitudmatricula.conte.org.co:8080/consulta-matricula',
                    'https://conaltel.org/consulta-de-matriculados/','https://www.consejoprofesional.org.co/registro-proceso-consulta.php','http://servicios.minminas.gov.co/diie/Account/Login.aspx',
                    'http://www.cgm.gov.co/cgm/Paginaweb/IP/Paginas/Informes-de-Auditor%C3%ADa.aspx?RootFolder=%2Fcgm%2FPaginaweb%2FIP%2FInformes%20Auditora%2FInformes%20de%20Auditor%C3%ADa%20PGA%202019&FolderCTID=0x012000335E7AE28794B4459957B0A2D7D060C9&View=%7B47EE697C%2D3923%2D4F85%2DAB19%2D793383221337%7D',
                    'https://medellin.gestiontransparente.com/Rendicion/inicio.aspx','https://cu.epm.com.co/institucional/afinia','https://normograma.mintic.gov.co/mintic/docs/resolucion_mintic_1519_2020.html',
                    'https://mangakakalot.tv/','https://bit.ly/2PU0OFh']

datosarchivorestric=openfile.values
links_restringidos=agregar_datos(datosarchivorestric,links_restringidos)
print('Datos restringidos leidos')
links_restringidos.append("javascript:__doPostBack('dnn$dnnSearch$cmdSearch','')")
links=[se.get('href') for se in atributo]
links=quitar_restriciones(links,links_restringidos)
print('Restricciones eliminados de los links')
#Se lee archivo de restricciones y anexa#
print('Tamaño con duplicados:  '+ str(len(links)))
links=quitar_duplicados_lista(links)
print('Tamaño sin duplicados:  '+ str(len(links)))
#[atribu.get('href') for atribu in links if atribu.get('href').startswith('/') ==True ]
print('Arreglando rutas relativas')
links=arreglar_relativas(links)
print('Rutas relativas arregladas')
#impresion_lista(links)






#-----Links de segundo nivel-----#
print('----------------------Comienza segundo nivel----------------------')
#----------------------Funciones segundo nivel----------------------#
links_2=[]
def links_segundo_nivel(links_entra):
  link_2_nivel=[]
  pagiinter=''
  for i in links_entra:
    j=links_entra.index(i)
    print(links_entra[j])
    if j<len(links_entra):
      #links=links_entra[j]
      try:
        pagiinter=requests.get(links_entra[j])
        if pagiinter.status_code==200:
          #print(pagiinter.status_code)
          sopa=BeautifulSoup(pagiinter.text,'lxml')
          atributo3=sopa.findAll('a')
          listax=[se.get('href') for se in atributo3]
          link_2_nivel=agregar_datos(listax,link_2_nivel)
          print('Tamaño de la lista acumuladora: ' + str(len(link_2_nivel)))
      except Exception as e:
        print('Error')
        print(e)
        print('\n')
      #print(links2 + '  codigo:'  + str(pagiinter.status_code)+ '  posición ' + str(j))
      int(j)
  return link_2_nivel

links_2=links_segundo_nivel(links)

print('Termino el recorrido')
links_2=quitar_restriciones(links_2,links_restringidos)
print('Quitar restricciones 2')
print("Sin quitar los java: "+str(len(links_2)))
links_2=validar_prefijos_lista_a_dato(links_2,lista_prefijos)
print("Quitados de prefijos: " + str(len(links_2)))
links_2=validar_no_contenga_lista_a_dato(links_2,lista_no_debe_contener)
print("Quitados de lo que no debe contener : " + str(len(links_2)))
links_2=quitar_duplicados_lista(links_2)
print(len(links_2))
links_2=arreglar_relativas(links_2)
impresion_lista(links_2)

print('----------------------Comienza tercer nivel----------------------')
print('----------------------Comienza tercer nivel----------------------')
print('----------------------Comienza tercer nivel----------------------')

#----------------------Funciones tercer nivel----------------------
links_3=links_segundo_nivel(links_2)
print('Termino el recorrido segundo nivel')
links_3=quitar_restriciones(links_3,links_restringidos)
print('Quitar restricciones 3')
print("Sin quitar los java: "+str(len(links_3)))
links_3=validar_prefijos_lista_a_dato(links_3,lista_prefijos)
print("Quitados de prefijos: " + str(len(links_3)))
links_3=validar_no_contenga_lista_a_dato(links_3,lista_no_debe_contener)
print("Quitados de lo que no debe contener : " + str(len(links_3)))
links_3=quitar_duplicados_lista(links_3)
print(len(links_3))
links_3=arreglar_relativas(links_3)
impresion_lista(links_3)





#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#
#------------------------------------------------------------------------------#

#Pasar los links a una hoja de excel#
libro=Workbook()
sheet=libro.active
#sheet['B1']='Item'
sheet['A1']= 'Link'
fila=2 #Fila inicia
col_uno=1 #Columnnas de links
col_dato=2

for dato in links:
  #sheet.cell(column=col_uno,row=fila, value=fila-1)
  sheet.cell(column=col_uno,row=fila,value=dato)
  fila=fila+1

libro.save('prueba1.xlsx')
path1="/content/prueba1.xlsx"
openfile1=pd.read_excel(path1,engine='openpyxl',sheet_name='Sheet')
datosarchivolec=openfile1.values


Datos restringidos leidos
Restricciones eliminados de los links
Tamaño con duplicados:  162
Se eliminaron : 24 duplicados
Tamaño sin duplicados:  138
Arreglando rutas relativas
Cambios realizados: 5
Rutas relativas arregladas
----------------------Comienza segundo nivel----------------------
https://www.cens.com.co/transparencia-y-acceso-a-la-informacion-publica-2021
Tamaño de la lista acumuladora: 335
https://www.cens.com.co/servicios-in-linea
Tamaño de la lista acumuladora: 345
https://www.cens.com.co/clientes-y-usuarios/tramites-y-servicios
Tamaño de la lista acumuladora: 623
https://www.cens.com.co/radica-tu-pqr
Tamaño de la lista acumuladora: 838
https://www.cens.com.co/participacion-ciudadana
Tamaño de la lista acumuladora: 1112
https://www.cens.com.co/
Tamaño de la lista acumuladora: 1353
https://www.cens.com.co/home/informes-de-sostenibilidad
Tamaño de la lista acumuladora: 1612
https://www.cens.com.co/home/eventos-y-patrocinios
Tamaño de la lista acumuladora: 1851
https://www.

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#------------------EXCEL-----------------#
#Librerias para pasar al Excel#
from openpyxl import load_workbook,Workbook
from openpyxl.styles import Font
from datetime import datetime

#Pasar los links a una hoja de excel#
libro=Workbook()
sheet=libro.active
sheet['A1']='Jordyn'
libro.save('prueba1.xlsx')
print('Se creo el archivo')
#--Leyendo nuevamente el archivo---#
path="/content/links_restringido.xlsx"
openfile=pd.read_excel(path,engine="openpyxl",sheet_name="Sheet")
datosarchivorestric=openfile.values
for i in datosarchivorestric:
  print(i)

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 150  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [ ]:
from IPython.utils.text import string
links_restringidos=['#','',None,'https://gobiernodigital.mintic.gov.co/portal/','https://www.tiktok.com/@censgrupoepm',
                    'https://www.linkedin.com/company/64512425/admin/','https://www.youtube.com/user/buenasenergiascens' ,
                    'https://twitter.com/CENSGrupoEPM?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor' ,
                    'https://www.instagram.com/censgrupoepm/?hl=es-la','https://apps.apple.com/co/app/cens-app-movil/id1357634972',
                    'https://play.google.com/store/apps/details?id=com.cens.app.movil&hl=es_CO&gl=US','018000522955','018000414115',
                    '(607)5824444','115','javascript:void(0);','javascript:void(0)','https://www.gov.co/home/','https://www.colombia.co/',
                    'https://gobiernodigital.mintic.gov.co/portal','/institucional/transparencia/normatividad','/terminos-y-condiciones',
                    '/institucional/proteccion-de-datos-personales']

for i in links_restringidos:
  if str(i).startswith('#'):
    print(i)

#
